<a href="https://colab.research.google.com/github/xBosse/notebooks/blob/main/Datenerweiterung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INSTALL IMPORT

In [ ]:
pip install spacy

In [ ]:
!git clone https://github.com/xBosse/datenerweiterung.git

Cloning into 'datenerweiterung'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 109 (delta 11), reused 0 (delta 0), pack-reused 63
Receiving objects: 100% (109/109), 8.64 MiB | 7.40 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9 MB 6.8 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=24d0ae84dc8d000e610b5bae2ef305c894834eecb22a0be64c112ff1163a399b
  Stored in directory: /tmp/pip-ephem-wheel-cache-4d82hg7g/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
pip install lxml

In [ ]:
pip install bs4

In [ ]:
###########################
def list_to_string(list):
  string = ""
  for x in list:
    string = string + str(x) + "\n"
  return string


# Subj Expr Lexicon 

In [ ]:
# Reading as Txt file

with open('datenerweiterung/coarse_grained_opinion_role_lexicon_verbs_german.txt') as f:
    lines = f.readlines()
# Closing file
f.close()


newlines = []
for line in lines:
  a = line.split("\t")
  newlines.append(a[0])


  lex_list = newlines
lex = list_to_string(newlines)


#write to file
text_file = open("lexicon.txt", "w")
text_file.write(lex)
text_file.close()

# Polit. Texte parsen / BeautifulSoup XML

In [ ]:
from bs4 import BeautifulSoup

klassen = ["J_1","J","O"]
texte = []

path1 = "datenerweiterung/"
path2 = "-data.xml"

for x in range(37):
  path = path1 + str(x+20001) + path2

  with open(path, 'r') as f:
      contents = f.read()
      soup = BeautifulSoup(contents, 'xml')

  for k in klassen:
    for line in soup.find_all(klasse=k):
      #print(line.text)
      texte.append(line.text)  

 

In [ ]:
len(texte)

51285

# Texte Tokenizen SPACY

In [ ]:
import spacy.cli 
spacy.cli.download("de_core_news_sm")

import spacy
from spacy.lang.de.examples import sentences 

nlp = spacy.load("de_core_news_sm")

#####   Bsp.   #####
#doc = nlp(sentences[0])
#print(doc.text)
#for token in doc:
   # print(token.text, token.pos_, token.dep_)

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
texte_tokens = []
for line in texte:
  doc = nlp(line)
  line_tokens = []
  for token in doc:
    line_tokens.append(token.text)
  texte_tokens.append(line_tokens)

# Filter/Labeling

Filtering

In [ ]:
def match(x):
    for lex in lex_list:
      if lex in x:
        return True
    return False

In [ ]:
texte_filtered = []
for line in texte_tokens:
  if match(line):
    texte_filtered.append(line)

print("Filtered:", len(texte_filtered))
print("Unfiltered:", len(texte))




Filtered: 15755
Unfiltered: 51285


Labeling

In [ ]:
texte_label = []
for line in texte_filtered:
  line_label = []
  for word in line:
    if match(word):
      line_label.append("1")
    else:
      line_label.append("0")
  texte_label.append(line_label)


In [ ]:
### STATS

aa = 0
for x in range(len(texte_label)):
  for y in range(len(texte_label[x])):
    if texte_label[x][y] == "1":
      aa += 1

print(aa)
print(len(texte_label))

28702
15755


#Formatting Noisy-labeled Data to Json

In [ ]:
# IN GLEICHES FORMAT WIE SHARED TASK DATEN BRINGEN

# {"words": ["Im", "Jahr", "2000", "hat", "sich", "die", "Welt", "den", "Uno-Millenniumszielen", "verpflichtet", ",", "auch", "die", "Schweiz", "hat", "diese", "Ziele", "unterzeichnet", "."], "subj": [0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0]}

texte_json = []


for x in range(len(texte_label)):
  words = "{\"words\": ["
  labels = "], \"subj\": ["

  for y in range(len(texte_label[x])):
    l = texte_label[x][y]
    w = texte_filtered[x][y]
    labels += str(l) + ","
    words += "\"" + w + "\", "

  texte_json.append(words[:-2] + labels[:-1] + "]}")

# Train Test Val Splits

In [ ]:
texte_json[0]

'{"words": ["Herr", "Präsident", "!", "Sehr", "geehrte", "Damen", "und", "Herren", "!", "Heute", "ist", "in", "der", "Tat", "ein", "besonderer", "Tag", "für", "die", "Demokratie", ".", "Nach", "der", "freien", "Wahl", "des", "deutschen", "Volkes", "stehen", "wir", "heute", "hier", "als", "Abgeordnete", "des", "Deutschen", "Bundestages", ",", "um", "uns", "als", "Bundestag", "zu", "konstituieren", "."], "subj": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}'

In [ ]:
import random

def train_val_test_splits(lines):
  random.shuffle(lines)

  cutoff = int(len(lines) * 0.8)
  cutoff2 = int(len(lines) * 0.1)
  lines_train = lines[:cutoff]
  lines_val = lines[cutoff+cutoff2:]
  lines_test = lines[cutoff:cutoff+cutoff2]

  print(len(lines),cutoff,cutoff2)

  print("Train: " + str(len(lines_train)) + " Val: " + str(len(lines_val)) + " Test: " + str(len(lines_test)))


  json_string = list_to_string(lines_train)  
  text_file = open("train_texte.txt", "w")
  text_file.write(json_string)
  text_file.close()

  json_string = list_to_string(lines_val)    
  text_file = open("val_texte.txt", "w")
  text_file.write(json_string)
  text_file.close()

  json_string = list_to_string(lines_test)    
  text_file = open("test_texte.txt", "w")
  text_file.write(json_string)
  text_file.close()


  json_string = list_to_string(texte_json) 
  text_file = open("texte_json.txt", "w")
  text_file.write(json_string)
  text_file.close()


In [ ]:
train_val_test_splits(texte_json)


15755 12604 1575
Train: 12604 Val: 1576 Test: 1575
